In [1]:
import os
import sys
curruser = os.environ.get('USER')

_labdata = os.environ.get("LABDATA_PYSPARK")
sys.path.insert(0, _labdata)

if curruser in os.listdir("/opt/workspace/"):
    sys.path.insert(0, '/opt/workspace/{user}/notebooks/support_library/'.format(user=curruser))
    sys.path.insert(0, '/opt/workspace/{user}/libs/python3.5/site-packages/'.format(user=curruser))
#     sys.path.insert(0, '/opt/workspace/{user}/notebooks/labdata/'.format(user=curruser))
else:
    sys.path.insert(0, '/home/{}/notebooks/support_library/'.format(curruser))
    sys.path.insert(0, '/home/{}/python35-libs/lib/python3.5/site-packages/'.format(curruser))
    # sys.path.insert(0, '/home/{}/notebooks/labdata/lib/'.format(curruser))

#import tendo.singleton
import warnings
warnings.filterwarnings('ignore')

import joblib
import json
from joblib import Parallel, delayed

from time import sleep
from itertools import islice
from multiprocessing import Pool, Process, JoinableQueue
from multiprocessing.pool import ThreadPool
from functools import partial
import subprocess
from threading import Thread
import time
from datetime import datetime as dt

from transliterate import translit

from lib.spark_connector import SparkConnector
from lib.sparkdb_loader import *
from lib.connector import OracleDB
import pyspark
from pyspark import SparkContext, SparkConf, HiveContext
from pyspark.sql.window import Window
from pyspark.sql.functions import *
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.sql.dataframe import DataFrame

import re
import pandas as pd
import numpy as np
from tqdm._tqdm_notebook import tqdm_notebook
from pathlib import Path
import shutil
import loader as load
from collections import ChainMap

from lib.config import *
from lib.tools import *

# sing = tendo.singleton.SingleInstance()

# os.chdir('/opt/workspace/ektov1-av_ca-sbrf-ru/notebooks/Clickstream_Analytics/AutoUpdate/')
# os.chdir('/opt/workspace/{}/notebooks/clickstream/AutoUpdate/'.format(curruser))

def show(self, n=10):
    return self.limit(n).toPandas()

def typed_udf(return_type):
    '''Make a UDF decorator with the given return type'''

    def _typed_udf_wrapper(func):
        return f.udf(func,return_type)

    return _typed_udf_wrapper

pyspark.sql.dataframe.DataFrame.show = show

def print_and_log(message: str):
    print(message)
    logger.info(message)
    return None


In [2]:
def map_product_group(p_crm_prod, b_rdd, cols):

    import pandas as pd

    if (p_crm_prod is not None) and (p_crm_prod not in ('', ' ')):
        v_crm_prod = p_crm_prod.upper()
        if v_crm_prod in ('ЭКСПРЕСС ОВЕРДРАФТ','ОВЕРДРАФТНОЕ КРЕДИТОВАНИЕ',
                          'ОБОРОТНОЕ КРЕДИТОВАНИЕ', 'КРЕДИТ'):
            v_crm_prod =  'КРЕДИТОВАНИЕ'
        elif v_crm_prod in ('ДИСТАНЦИОННОЕ БАНКОВСКОЕ ОБСЛУЖИВАНИЕ'):
            v_crm_prod =  'ДИСТАНЦИОННОЕ ОБСЛУЖИВАНИЕ';
        elif v_crm_prod in ('ОТКРЫТИЕ СЧЕТА В ВАЛЮТЕ','ОТКРЫТИЕ СЧЕТА В РУБЛЯХ',
                            'СПЕЦСЧЕТА','РКО','РАСЧЕТНО-КАССОВОЕ ОБСЛУЖИВАНИЕ'):
            v_crm_prod = 'РКО'
        elif v_crm_prod in ('ПРОДУКТЫ ИНКАССАЦИИ'):
            v_crm_prod =  'ИНКАССАЦИЯ'
        elif v_crm_prod in ('НЕСНИЖАЕМЫЙ ОСТАТОК'):
            v_crm_prod = 'ПРОДУКТЫ ПРИВЛЕЧЕНИЯ СРЕДСТВ';
        elif 'ПАКЕТ УСЛУГ' in v_crm_prod:
            v_crm_prod = 'ПАКЕТЫ УСЛУГ'
        elif v_crm_prod in ('ВЭД И ВАЛЮТНЫЙ КОНТРОЛЬ'):
            v_crm_prod = 'ВАЛЮТНЫЙ КОНТРОЛЬ'
        elif v_crm_prod in ('АККРЕДИТИВЫ'):
            v_crm_prod = 'ДОКУМЕНТАРНЫЕ ОПЕРАЦИИ'
        elif v_crm_prod in ('КОРПОРАТИВНАЯ КАРТА'):
            v_crm_prod = 'БИЗНЕС-КАРТА'
        elif v_crm_prod in ('БИЗНЕС-КАРТА'):
            v_crm_prod = 'БИЗНЕС-КАРТА'
        elif v_crm_prod in ('ДЕПОЗИТЫ (СРОЧНЫЕ)'):
            v_crm_prod = 'ПРОДУКТЫ ПРИВЛЕЧЕНИЯ СРЕДСТВ'
        elif v_crm_prod in ('АКТИВИЗАЦИЯ РКО'):
            v_crm_prod = None

        dct = {col: [b_rdd.value[i][col] for i in range(len(b_rdd.value))] for col in cols}
        df = pd.DataFrame.from_dict(dct, orient='columns')
        if v_crm_prod is not None:
            try:
                v_prod_group_trg = df.loc[df['P_NAME'].str.contains(v_crm_prod),'P_GROUP'].tolist()[0]
                return v_prod_group_trg.lower()
            except:
                v_prod_group_trg = v_crm_prod
                return v_prod_group_trg.lower()
        else:
            return None
    else:
        return None

def udf_map_product_group(b_rdd, cols):
    return f.udf(lambda p: map_product_group(p, b_rdd, cols))


In [3]:
CONN_SCHEMA='sbx_team_digitcamp'

In [4]:
%%time
sp = spark(schema=CONN_SCHEMA,
           dynamic_alloc=False,
           numofinstances=10,
           numofcores=8,
           executor_memory='40g',
           driver_memory='40g',
           kerberos_auth=True,
           process_label="SAS_ANALYT_"
           )

hive = sp.sql
print(sp.sc.version)

====================================================== 2021-10-29 ======================================================
# __init__ : begin
2.4.0.cloudera2
CPU times: user 68.7 ms, sys: 52.8 ms, total: 121 ms
Wall time: 36 s


In [23]:
ld=load.Loader(init_dsn=True, encoding='cp1251',  sep=',')
db = load.OracleDB('tech_iskra[iskra]', 'Uthvfy123', ld._get_dsn('iskra4')) 
db.connect()
curs = db.cursor

In [24]:
table_name = 'mw_atb_segmen_prod_group'
query = '''
select /*+ parallel (12) */
          *
FROM {}
--where rownum < 1000
'''.format(table_name)

df = pd.read_sql(query, con=db.connection)

In [101]:
df.loc[df['P_NAME'].str.contains('ОВЕРДР'),['P_NAME', 'P_GROUP']]

,P_NAME,P_GROUP
83,ИНДИВИДУАЛЬНЫЙ ОВЕРДРАФТ,КРЕДИТОВАНИЕ
84,ИНДИВИДУАЛЬНЫЙ ОВЕРДРАФТ-ПРОТО,КРЕДИТОВАНИЕ
289,ОВЕРДРАФТ,КРЕДИТОВАНИЕ
290,ОВЕРДРАФТ (СТРУКТУРИРОВАННЫЙ),КРЕДИТОВАНИЕ
291,ОВЕРДРАФТ С ОБЩИМ ЛИМИТОМ,КРЕДИТОВАНИЕ
292,ОВЕРДРАФТ С ОБЩИМ ЛИМИТОМ-ПРОТО,КРЕДИТОВАНИЕ
312,СМАРТ-ОВЕРДРАФТ (МАЛЫЙ),КРЕДИТОВАНИЕ
391,ЭКСПРЕСС-ОВЕРДРАФТ,КРЕДИТОВАНИЕ
392,ЭКСПРЕСС-ОВЕРДРАФТ-ПРОТО,КРЕДИТОВАНИЕ
406,(МБ) БИЗНЕС-ОВЕРДРАФТ,КРЕДИТОВАНИЕ


In [105]:
set(df['P_GROUP'].values.tolist())

{'CASH MANAGEMENT',
 'АРЕНДА ПОМЕЩЕНИЙ',
 'АРЕНДА СЕЙФОВ',
 'БАНКОВСКАЯ ГАРАНТИЯ',
 'БАНКОВСКОЕ СОПРОВОЖДЕНИЕ КОНТРАКТОВ',
 'ВАЛЮТНЫЙ КОНТРОЛЬ',
 'ВЕКСЕЛЯ',
 'ДЕЛОВАЯ СРЕДА',
 'ДЕПОЗИТАРНЫЕ УСЛУГИ',
 'ДЕПОЗИТНЫЕ СЕРТИФИКАТЫ',
 'ДЕПОЗИТЫ',
 'ДИСТАНЦИОННОЕ ОБСЛУЖИВАНИЕ',
 'ДОКУМЕНТАРНЫЕ ОПЕРАЦИИ',
 'ДОХОДЫ',
 'ЗАРПЛАТНЫЕ ПРОЕКТЫ',
 'ИНВЕСТИЦИОННЫЕ ПРОДУКТЫ',
 'ИНКАССАЦИЯ',
 'ИНФОРМАЦИОННЫЕ УСЛУГИ',
 'КОНВЕРСИОННЫЕ ОПЕРАЦИИ',
 'КОНСУЛЬТИРОВАНИЕ ПО IPO',
 'КОНСУЛЬТИРОВАНИЕ ПО M&A',
 'КОРПОРАТИВНАЯ КАРТА',
 'КРЕДИТОВАНИЕ',
 'ЛИЗИНГ',
 'НПФ СБЕРБАНКА',
 'ПАКЕТЫ УСЛУГ',
 'ПРИЕМ ПЛАТЕЖЕЙ ФИЗ. ЛИЦ',
 'ПРОДУКТЫ МБ',
 'ПРОДУКТЫ ПРИВЛЕЧЕНИЯ СРЕДСТВ',
 'РАЗМЕЩЕНИЕ ОБЛИГАЦИЙ',
 'РАСХОДЫ',
 'РКО',
 'РОЗНИЧНЫЕ ПРОДУКТЫ',
 'САМОИНКАССАЦИЯ',
 'СБЕРБАНК АСТ',
 'СИСТЕМА ДИСТАНЦИОННОГО ОБСЛУЖИВАНИЯ',
 'СТРАХОВАНИЕ',
 'ТАРИФНЫЕ ПЛАНЫ',
 'ТЕХНИЧЕСКИЕ ПРОДУКТЫ',
 'ТОРГОВОЕ ФИНАНСИРОВАНИЕ',
 'УСТАНОВКА БАНКОМАТОВ',
 'ФАКТОРИНГ',
 'ЭКВАЙРИНГ'}

In [106]:
len(set(df['P_GROUP'].values.tolist()))

43

In [10]:
mw_prod_group = hive.sql('''select * from {}.mw_atb_segmen_prod_group'''.format(CONN_SCHEMA))
df = mw_prod_group.take(sdf.count())
b_rdd = sp.sc.broadcast(df)
cols = mw_prod_group.columns

In [11]:
mw_prod_group.show()

,P_NAME,P_GROUP
0,ПОСТЭКСП. ФИНАНСИР. ПО АККРЕДИТИВАМ ОАО «БПС-Б...,КРЕДИТОВАНИЕ
1,ПОСТЭКСПОРТНОЕ ФИНАНСИРОВАНИЕ,КРЕДИТОВАНИЕ
2,ПОСТЭКСПОРТНОЕ ФИНАНСИРОВАНИЕ (СТРУКТУРИРОВАННЫЙ),КРЕДИТОВАНИЕ
3,ПОСТЭКСПОРТНОЕ ФИНАНСИРОВАНИЕ (УНИФИЦИРОВАННЫЙ),КРЕДИТОВАНИЕ
4,ПРЕДЗАЯВКА В СБЕРБАНК ЛИЗИНГ,КРЕДИТОВАНИЕ
5,ПРЕДЗАЯВКА В СБЕРБАНК ЛИЗИНГ,ЛИЗИНГ
6,ПРЕДСЕЗОННОЕ КРЕДИТОВАНИЕ ЮР.ЛИЦ – НЕДРОПОЛЬЗО...,КРЕДИТОВАНИЕ
7,ПРЕДСЕЗОННОЕ ФИНАНСИРОВАНИЕ НЕДРОПОЛЬЗОВАТЕЛЕЙ...,КРЕДИТОВАНИЕ
8,ПРЕДЭКСПОРТНОЕ ФИНАНСИРОВАНИЕ,КРЕДИТОВАНИЕ
9,ПРЕДЭКСПОРТНОЕ ФИНАНСИРОВАНИЕ (СТРУКТУРИРОВАННЫЙ),КРЕДИТОВАНИЕ


In [28]:
mw_prod_group.filter("lower(P_NAME) like '%договор%'").collect()

[Row(P_NAME='ГАРАНТИИ ИСПОЛН. ФИН. ОБЯЗАТ-В ПО ДОГОВОРУ/КОНТРАКТУ - С ОФОРМЛ. ОБЕСПЕЧЕНИЯ В ВИДЕ ЗАЛОГА ВЕКСЕЛЕЙ', P_GROUP='БАНКОВСКАЯ ГАРАНТИЯ'),
 Row(P_NAME='ГАРАНТИИ ИСПОЛНЕНИЯ ОБЯЗ-В ПО ДОГОВОРУ/КОНТРАКТУ В РАМКАХ ПАКЕТНОГО ПРЕДЛОЖЕНИЯ', P_GROUP='БАНКОВСКАЯ ГАРАНТИЯ'),
 Row(P_NAME='ГАРАНТИИ ИСПОЛНЕНИЯ ФИНАНСОВЫХ ОБЯЗАТЕЛЬСТВ ПО ДОГОВОРУ (КРОМЕ КРЕДИТНОГО ДОГОВОРА)', P_GROUP='БАНКОВСКАЯ ГАРАНТИЯ'),
 Row(P_NAME='ГАРАНТИИ ИСПОЛНЕНИЯ ФИНАНСОВЫХ ОБЯЗАТЕЛЬСТВ ПО ДОГОВОРУ (КРОМЕ КРЕДИТНОГО ДОГОВОРА)', P_GROUP='КРЕДИТОВАНИЕ'),
 Row(P_NAME='ГАРАНТИИ ИСПОЛНЕНИЯ ФИНАНСОВЫХ ОБЯЗАТЕЛЬСТВ ПО КРЕДИТНОМУ ДОГОВОРУ', P_GROUP='БАНКОВСКАЯ ГАРАНТИЯ'),
 Row(P_NAME='ГАРАНТИИ ИСПОЛНЕНИЯ ФИНАНСОВЫХ ОБЯЗАТЕЛЬСТВ ПО КРЕДИТНОМУ ДОГОВОРУ', P_GROUP='КРЕДИТОВАНИЕ'),
 Row(P_NAME='ГАРАНТИИ/КОНТРГАРАНТИИ ДОГОВОРНАЯ  (ВОЗВРАТА АВАНСА+ ГАРАНТИЯ КАЧЕСТВА)', P_GROUP='БАНКОВСКАЯ ГАРАНТИЯ'),
 Row(P_NAME='ГАРАНТИИ/КОНТРГАРАНТИИ ДОГОВОРНАЯ (ВОЗВРАТА АВАНСА)', P_GROUP='БАНКОВСКАЯ ГАРАНТИЯ'),
 Row(P_NAME='ГАРАНТИИ/КОНТРГАРАН

In [73]:
# conn_schema = 'sbx_team_digitcamp'
table_name = 'MA_CMDM_MA_AGREEMENT' #'MA_CMDM_MA_DEAL_NEW'

In [5]:
agrmnt = hive.sql('''select distinct FIN_AGRMNT_TYPE_DESC from sbx_team_digitcamp.MA_CMDM_MA_AGREEMENT''').collect()

In [13]:
fin_agrmnt_type = [col['FIN_AGRMNT_TYPE_DESC'] for col in agrmnt]
fin_agrmnt_type

['Зарплатный договор',
 'Сделка с неснижаемым остатком',
 'Аккредитив',
 'Счета суррогатных кредитных договоров',
 'Депозит юридического лица',
 'Договор НОСТРО',
 'Самоинкассация',
 'СББОЛ',
 'Счета договора торгового финансирования',
 'Сейфовые ячейки (Аренда сейфов)',
 'Кредитная линия',
 'Бизнес-карта',
 'Договор о предоставлении гарантии',
 'Прочие виды ссудной и приравнённой к ней задолженности',
 'Генеральное соглашение по аккредитиву',
 'Договор РКО',
 'Генеральное соглашение по овердрафту',
 'Транш',
 'Факторинг',
 'Банковская гарантия (выданная)',
 'Инкассация (включая самоинкассацию)',
 'Депозитный сертификат',
 'Эквайринг',
 'Рамочное генеральное соглашение',
 'Вексель',
 'Счета суррогатных непокрытых аккредитивов',
 'Овердрафт ЛОРО',
 'Кредитный договор']

## Analyse U_BASE_DEAL

In [6]:
u_base_deals = hive.table("cib_custom_cib_ml360.u_base_deals")

In [7]:
udeals_innprod_uniq = u_base_deals.select("inn", "product_id").distinct().collect()
udeal_dct = [row.asDict() for row in udeals_innprod_uniq]
udeal_df = pd.DataFrame.from_dict(udeal_dct)

In [7]:
udeal_df.head()

,inn,product_id
0,7723887926,1-1GW0
1,7811210812,1-1GT0
2,2721213312,1-115HHZC
3,7017299536,1-115HI62
4,010510290631,1-1OA43KI


In [8]:
gr_cols = \
["inn", 
 "product_id"
]

udeals_gr = \
u_base_deals.groupBy(gr_cols).agg(f.countDistinct("product_id").alias('cnt'))\
                    .select(gr_cols+['cnt']).orderBy(f.col('cnt').desc()).toPandas()

In [9]:
udeals_gr.sort_index(ascending=False)

,inn,product_id,cnt
31726979,3851008997,1-115EQY8,1
31726978,2118002831,1-2M747AH,1
31726977,7733842843,1-OCYGMDT,1
31726976,032102284160,1-DAFHYND,1
31726975,366219434704,1-115HHYO,1
31726974,7705119583,1-XNQFAFZ,1
31726973,690607970250,1-115HHZC,1
31726972,6683004288,1-XNQFAFZ,1
31726971,422375346614,1-OCYF9VD,1
31726970,742004916226,1-IE1667Y,1


## MA_CMDM_MA_DEAL_NEW

In [ ]:
conn_schema = 'sbx_team_digitcamp'
table_name = 'MA_CMDM_MA_DEAL_NEW'

startdt = dt.now()

#/////////////////////////////////////////////////////////////////////////////////////
descr = hive.sql("describe extended {}.{}".format(conn_schema,table_name)).collect()
hasPartitioned = len([item.asDict() for item in descr if item.asDict()['col_name'] =='# Partition Information']) > 0
if hasPartitioned:
    try:
        parts = hive.sql("show partitions {}.{}".format(conn_schema,table_name)).collect()
        max_part = sorted(parts,reverse=True)[0]['partition']
        extract_date=re.compile("\d{4}\-\d{2}\-\d{2}")
        ext = extract_date.search(max_part)
        try:
            max_trunc_dt = ext.group(0)
        except:
            max_trunc_dt = None
    except (AnalysisException, IndexError):
        max_trunc_dt = None
else:
    max_trunc_dt = None
#/////////////////////////////////////////////////////////////////////////////////////

if max_trunc_dt is not None:
  sql = '''select max(create_dt) from {}.{} where creation_month = '{}' '''.format(conn_schema,table_name,max_trunc_dt)
else:
  sql = '''select max(create_dt) from {}.{}'''.format(conn_schema,table_name)

max_dt = hive.sql(sql).collect()
max_dt = max_dt[0]['max(create_dt)']
max_resp_dt_str = dt.strftime(max_dt, format='%Y-%m-%d %H:%M:%S.%f')

enddt = dt.now() - startdt

_str = max_resp_dt_str #json.dumps(_dct, ensure_ascii=False, indent=3)

print("total calculation time: {}".format(enddt.total_seconds()))
print("max(create_dt): {}".format(_str))

In [5]:
query = \
'''
select 
        distinct
        --inn, 
        --create_dt,
        --complete_dt,
        host_prod_id
        --close_reason_type_name,
        --appl_stage_name,
        --appl_type_name,
        --channel_mmb_ms,
        --channel_mmb_ckr,
        --channel_mmb_mkk,
        --channel_dzo
        --creation_month
from sbx_team_digitcamp.MA_CMDM_MA_DEAL_NEW
'''

In [6]:
%%time
macmdm_deal_prod_uniq = hive.sql(query).toPandas()
# macmdm_deal_innprod_uniq = macmdm_deal.select("inn", "host_prod_id").distinct().collect()

CPU times: user 8.65 s, sys: 676 ms, total: 9.33 s
Wall time: 52min 15s


In [35]:
sp.sc.stop()

In [28]:
macmdm_deal_prod_uniq = macmdm_deal_prod_uniq.fillna('-1')

In [22]:
macmdm_deal_prod_uniq.host_prod_id.value_counts()

1-4DQOXQ3R    1
1-3OUAX1R5    1
1-3OVPL6TD    1
1-1944KRBF    1
1-450VQ31Q    1
             ..
1-2GSLH9DP    1
1-125HLSTD    1
1-1TNDZ5F5    1
1-1MJ1FPTQ    1
1-1RRKBUQO    1
Name: host_prod_id, Length: 1522305, dtype: int64

In [19]:
prod_cnt = \
macmdm_deal_prod_uniq.groupby('host_prod_id')\
                        .agg(cnt=('host_prod_id', 'count')).sort_values('cnt', ascending=False)

In [20]:
prod_cnt.head(20)

,cnt
host_prod_id,
1-10000KFS,1
1-32MQOFY8,1
1-32MROA17,1
1-32MRK590,1
1-32MRJHUW,1
1-32MRFWP0,1
1-32MRENNF,1
1-32MRBGLN,1
1-32MR21WR,1


In [17]:
# madeal_dct = [row.asDict() for row in macmdm_deal_innprod_uniq]
# madeal_df = pd.DataFrame.from_dict(madeal_dct)

In [18]:
# madeal_df.rename(columns={'host_prod_id': 'product_id'}, inplace=True)

In [31]:
# madeal_df.describe()

,product_id,inn
count,31235987,31260154
unique,1522305,4073322
top,1-115HHZC,4002599884
freq,2780493,496


In [32]:
udeal_df.describe()

,inn,product_id
count,31726980,31726980
unique,4649651,559
top,,1-115HHZC
freq,133,3238535


In [ ]:
madeal_df['label'] = 'ma_deal'
out = udeal_df.merge(madeal_df, on=['inn', 'product_id'], suffixes=('_madeal', '_udeal'), how='outer')

In [46]:
out = out.drop_duplicates()

In [47]:
out.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35382580 entries, 0 to 35382579
Data columns (total 3 columns):
inn           object
product_id    object
label         object
dtypes: object(3)
memory usage: 1.1+ GB


In [ ]:
sum(out.label.isnull())

In [48]:
out.describe()

,inn,product_id,label
count,35380159,35355992,31262575
unique,4681192,1522340,1
top,4002599884,1-115HHZC,ma_deal
freq,496,3238932,31262575


In [ ]:
len(set(madeal_df.inn.values) - set(udeal_df.inn.values))

In [ ]:
len(set(madeal_df.product_id.values) - set(udeal_df.product_id.values))

## Join Deals with Agreements and Prod Dict

In [93]:
sql = \
'''
with deals as (
select 
        inn, 
        create_dt,
        complete_dt,
        host_prod_id,
        close_reason_type_name,
        appl_stage_name,
        --appl_type_name,
        --channel_mmb_ms,
        --channel_mmb_ckr,
        --channel_mmb_mkk,
        --channel_dzo
        creation_month
from sbx_team_digitcamp.MA_CMDM_MA_DEAL_NEW
where (create_dt is not Null) and
      (appl_stage_name like '%Закрыта/Заключена%') and
      (close_reason_type_name is Null)
),

prods as (
select 
       CRM_PRODUCT_ID, 
       PRODUCT_CRM_NM, 
       PRODUCT_CD_MMB,
       PRODUCT_FULL_NM,
       PRODUCT_SUBGROUP,
       PRODUCT_GROUP
       --CRM_ACTIVE_FLG
from sbx_team_digitcamp.ma_dict_v_product_dict
),

deals_prods as (
select 
    deals.inn,
    deals.create_dt,
    deals.complete_dt,
    deals.appl_stage_name,
    prods.PRODUCT_CRM_NM,
    prods.CRM_PRODUCT_ID,
    prods.PRODUCT_SUBGROUP
from deals
left join prods on deals.host_prod_id = prods.CRM_PRODUCT_ID
where 
(prods.PRODUCT_SUBGROUP = "Бизнес-карта") and 
(timestamp(deals.creation_month) >= timestamp('2021-01-01'))
),

agrmnts as(
select 
    AGREEMENT_ID,
    INN,
    ACCT_SIGNED_DT,
    ACCT_CLOSE_DT,
    FIN_AGRMNT_TYPE_DESC
from sbx_team_digitcamp.MA_CMDM_MA_AGREEMENT
where (FIN_AGRMNT_TYPE_DESC = 'Бизнес-карта') and
      (acct_signed_str is not Null) and
      (timestamp(acct_signed_str) between timestamp('2021-01-01') and timestamp('2022-01-01'))
)

select 
    dls.inn,
    dls.create_dt,
    dls.complete_dt,
    agrmnts.ACCT_SIGNED_DT,
    agrmnts.ACCT_CLOSE_DT,
    agrmnts.AGREEMENT_ID,
    dls.appl_stage_name,
    dls.PRODUCT_CRM_NM,
    dls.CRM_PRODUCT_ID,
    dls.PRODUCT_SUBGROUP 
from deals_prods dls
left join agrmnts on (dls.inn = agrmnts.INN) and
                     (abs(unix_timestamp(agrmnts.ACCT_SIGNED_DT) - 
                          unix_timestamp(dls.create_dt)) <= 30 * 24 * 60 * 60
                     ) and
                     (agrmnts.ACCT_SIGNED_DT < dls.create_dt)
where agrmnts.ACCT_SIGNED_DT is not Null
'''

In [94]:
hive.sql(sql).show(20)

,inn,create_dt,complete_dt,ACCT_SIGNED_DT,ACCT_CLOSE_DT,AGREEMENT_ID,appl_stage_name,PRODUCT_CRM_NM,CRM_PRODUCT_ID,PRODUCT_SUBGROUP
0,020504306907,2021-05-25 08:38:19,2021-05-26,2021-04-29,None,22833853805.0000000000,05.Закрыта/Заключена,Бизнес-карта,1-115HHYO,Бизнес-карта
1,0261012586,2021-06-08 11:13:13,2021-06-08,2021-06-04,None,23963908847.0000000000,05.Закрыта/Заключена,СМС-информирование по Бизнес-карте,1-OCYF9TN,Бизнес-карта
2,0261012586,2021-06-08 11:13:13,2021-06-08,2021-06-04,None,23963908847.0000000000,05.Закрыта/Заключена,Бизнес-карта,1-115HHYO,Бизнес-карта
3,026701962578,2021-03-19 10:41:47,2021-03-19,2021-03-19,None,21080310698.0000000000,05.Закрыта/Заключена,СМС-информирование по Бизнес-карте,1-OCYF9TN,Бизнес-карта
4,026701962578,2021-03-19 10:41:47,2021-03-19,2021-03-19,None,21080310698.0000000000,05.Закрыта/Заключена,Моментальная Бизнес-карта,1-WCK2VTA,Бизнес-карта
5,027300589061,2021-04-14 05:30:10,2021-04-14,2021-04-14,None,22345947274.0000000000,05.Закрыта/Заключена,Моментальная Бизнес-карта,1-WCK2VTA,Бизнес-карта
6,0274171030,2021-03-03 07:42:22,2021-03-03,2021-03-03,None,20650459579.0000000000,05.Закрыта/Заключена,Моментальная Бизнес-карта,1-WCK2VTA,Бизнес-карта
7,0274171030,2021-03-03 07:42:22,2021-03-03,2021-03-03,None,20650459579.0000000000,05.Закрыта/Заключена,СМС-информирование по Бизнес-карте,1-OCYF9TN,Бизнес-карта
8,027711397609,2021-04-23 10:05:05,2021-04-23,2021-04-23,None,22589160091.0000000000,05.Закрыта/Заключена,Моментальная Бизнес-карта,1-WCK2VTA,Бизнес-карта
9,0277954014,2021-02-19 08:08:44,2021-02-20,2021-02-17,None,20221911118.0000000000,05.Закрыта/Заключена,СМС-информирование по Бизнес-карте,1-OCYF9TN,Бизнес-карта


In [95]:
res = hive.sql(sql)

In [96]:
res.drop_duplicates().select('inn').distinct().count()

81149

In [30]:
prod_dict = hive.sql('''select CRM_PRODUCT_ID, 
                               PRODUCT_CRM_NM, 
                               PRODUCT_CD_MMB,
                               PRODUCT_FULL_NM,
                               PRODUCT_SUBGROUP,
                               PRODUCT_GROUP,
                               CRM_ACTIVE_FLG
                        from sbx_team_digitcamp.ma_dict_v_product_dict
                     ''')

In [32]:
prod_dict.filter("lower(PRODUCT_SUBGROUP) like '%бизнес%'").show(10)

,CRM_PRODUCT_ID,PRODUCT_CRM_NM,PRODUCT_CD_MMB,PRODUCT_FULL_NM,PRODUCT_SUBGROUP,PRODUCT_GROUP,CRM_ACTIVE_FLG
0,1-1DZP5W1R,Пилот. Бизнес-карта через ТхВ,None,None,Бизнес-карта,Бизнес-карта,1
1,1-164W2L8R,Трэвел бизнес карта,None,None,Бизнес-карта,Бизнес-карта,1
2,1-2LJG0UXU,Бизнес карта-Пластиковый носитель,None,None,Бизнес-карта,Бизнес-карта,1
3,1-2NARI42L,API. Переводы с бизнес-карт,None,API. Переводы с бизнес-карт,Бизнес-карта,Бизнес-карта,1
4,1-1DP7C8U0,Бизнес карта под ключ,None,"Пакет ""Бизнес-карта под ключ""",Бизнес-карта,РКО,0
5,1-HLPSULB,Кредитная бизнес-карта,CREDCORPCARD,Кредитная бизнес-карта,Кредитная бизнес-карта,Кредитование,1
6,1-2DBW9VNP,Бизнес-карта (оптовая продажа),None,Бизнес-карта (оптовая продажа),Бизнес-карта,Бизнес-карта,0
7,1-27TVRA5B,Банковское сопровождение контрактов,None,Банковское сопровождение для участников исполн...,Решения для бизнеса,Экосистема,1
8,1-13KV7EKC,Цифровая бизнес карта,CIFRBIZCARD,Цифровая бизнес-карта,Бизнес-карта,Бизнес-карта,1
9,1-ZYIAS1D,Бизнес-класс Google,None,None,Бизнес-карта,Бизнес-карта,1


In [76]:
parts = hive.sql("show partitions {}.{}".format(CONN_SCHEMA,table_name)).collect()
parts

[Row(partition='acct_signed_str=1800-01-01'),
 Row(partition='acct_signed_str=1800-10-01'),
 Row(partition='acct_signed_str=1818-09-01'),
 Row(partition='acct_signed_str=1882-04-01'),
 Row(partition='acct_signed_str=1899-12-01'),
 Row(partition='acct_signed_str=1900-01-01'),
 Row(partition='acct_signed_str=1900-02-01'),
 Row(partition='acct_signed_str=1900-03-01'),
 Row(partition='acct_signed_str=1900-05-01'),
 Row(partition='acct_signed_str=1900-06-01'),
 Row(partition='acct_signed_str=1900-09-01'),
 Row(partition='acct_signed_str=1900-10-01'),
 Row(partition='acct_signed_str=1900-11-01'),
 Row(partition='acct_signed_str=1901-01-01'),
 Row(partition='acct_signed_str=1901-02-01'),
 Row(partition='acct_signed_str=1901-05-01'),
 Row(partition='acct_signed_str=1901-09-01'),
 Row(partition='acct_signed_str=1902-01-01'),
 Row(partition='acct_signed_str=1902-07-01'),
 Row(partition='acct_signed_str=1902-08-01'),
 Row(partition='acct_signed_str=1902-09-01'),
 Row(partition='acct_signed_str=19

In [7]:
hive.sql('''select * from {}.{} where (inn = '2221191879') and 
                                      (acct_signed_str between '2021-02-01' and '2021-04-01')'''.format(CONN_SCHEMA, table_name)).show(40)

,AGREEMENT_ID,INN,KPP,ACCT_SIGNED_DT,CONTRACT_EXPIRATION_DT,ACCT_CLOSE_DT,FIN_AGRMNT_TYPE_DESC,ISO_CRNCY_CD,CRNCY_CD,AGREEMENT_NUM,ANALYTIC_CLOSE_DT,acct_signed_str
0,21818433411.0000000000,2221191879,222101001,2021-02-12 11:08:45,None,None,СББОЛ,None,None,285053786,None,2021-02-01
1,19908825428.0000000000,2221191879,222101001,2021-02-12 00:00:00,None,None,Договор РКО,RUR,810,40702810002000032928,None,2021-02-01
2,19994729343.0000000000,2221191879,222101001,2021-02-12 00:00:00,None,None,Самоинкассация,None,None,285053825,None,2021-02-01


In [9]:
deals = \
hive.sql(''' select 
                    inn, 
                    create_dt,
                    complete_dt,
                    host_prod_id,
                    --close_reason_type_name,
                    appl_stage_name
                    --appl_type_name,
                    --channel_mmb_ms,
                    --channel_mmb_ckr,
                    --channel_mmb_mkk,
                    --channel_dzo  
             from {}.{} where (inn = '2221191879') and
                              (creation_month between '2021-02-01' and '2021-04-01') '''.format(CONN_SCHEMA, "MA_CMDM_MA_DEAL_NEW"))

In [10]:
conditions = (prod_dict.CRM_PRODUCT_ID == deals.host_prod_id)
res = deals.join(prod_dict, on=conditions)

In [11]:
res.show(20)

,inn,create_dt,complete_dt,host_prod_id,appl_stage_name,CRM_PRODUCT_ID,PRODUCT_CRM_NM,PRODUCT_CD_MMB,PRODUCT_FULL_NM,PRODUCT_SUBGROUP,PRODUCT_GROUP,CRM_ACTIVE_FLG
0,2221191879,2021-02-11 06:05:05,None,1-115HHZC,06. Доставка документов,1-115HHZC,Открытие банковского счета с договором-констру...,RKO,Открытие расчетного счета,РКО,РКО,1
1,2221191879,2021-02-11 06:06:44,None,1-HLPSUHX,01.Ввод данных,1-HLPSUHX,ПУ Легкий старт,None,None,РКО,РКО,1
2,2221191879,2021-02-11 06:06:47,None,1-115HHZC,01.Ввод данных,1-115HHZC,Открытие банковского счета с договором-констру...,RKO,Открытие расчетного счета,РКО,РКО,1


In [15]:
deals = \
hive.sql(''' select 
                    inn, 
                    create_dt,
                    complete_dt,
                    host_prod_id,
                    --close_reason_type_name,
                    appl_stage_name
                    --appl_type_name,
                    --channel_mmb_ms,
                    --channel_mmb_ckr,
                    --channel_mmb_mkk,
                    --channel_dzo  
             from {}.{} where (inn = '2221191879') and
                              (creation_month in ('2021-02-01', 
                                                  '2021-03-01',
                                                  '2021-04-01',
                                                  '2021-05-01') )'''.format(CONN_SCHEMA, "MA_CMDM_MA_DEAL_NEW"))

In [16]:
conditions = (prod_dict.CRM_PRODUCT_ID == deals.host_prod_id)
res = deals.join(prod_dict, on=conditions)

In [ ]:
res.show()

In [173]:
startdt = dt.now()

#/////////////////////////////////////////////////////////////////////////////////////
descr = hive.sql("describe extended {}.{}".format(conn_schema,table_name)).collect()
hasPartitioned = len([item.asDict() for item in descr if item.asDict()['col_name'] =='# Partition Information']) > 0
if hasPartitioned:
    try:
        parts = hive.sql("show partitions {}.{}".format(conn_schema,table_name)).collect()
        max_part = sorted(parts, reverse=True)[0]['partition']
        extract_date=re.compile("\d{4}\-\d{2}\-\d{2}")
        ext = extract_date.search(max_part)
        try:
            max_trunc_dt = ext.group(0)
        except:
            max_trunc_dt = None
    except (AnalysisException, IndexError):
        max_trunc_dt = None
else:
    max_trunc_dt = None
#/////////////////////////////////////////////////////////////////////////////////////


In [174]:
if max_trunc_dt is not None:
  sql = '''select * from {}.{} where creation_month in 
                 ('2021-06-01', '2021-05-01',' 2021-04-01') '''.format(conn_schema, table_name)
else:
  sql = '''select * from {}.{}'''.format(conn_schema,table_name)

deals = hive.sql(sql)
# max_resp_dt_str = dt.strftime(max_dt, format='%Y-%m-%d %H:%M:%S.%f')

In [150]:
deals.select('appl_stage_name').distinct().show(20)

,appl_stage_name
0,12.Закрыта/Отказ
1,07.Подготовка документации
2,05. Подготовка документов по сделке
3,04.Тендерная процедура
4,04.2.Ожидание оформ. обеспеч.
5,Договор подписан
6,02.Предварительные переговоры
7,03.Подписание док-ии Клиентом
8,Закрыта/Заключена
9,None


## Cancelled Deals

In [270]:
deals_d = \
deals.filter((f.col('appl_stage_name').like("%Закрыта/Отказ%"))&
             (~f.isnull(f.col('close_reason_type_name')))&
             (~f.isnull(f.col('product_cd_mmb')))
            ) 
    
deals_d = deals_d.select('inn', 'product_cd_mmb', f.lit(1).alias('flg_deal_deny'))

deals_deny = deals_d.groupBy('product_cd_mmb').agg(f.count('flg_deal_deny').alias('cnt_deals_deny')).orderBy(f.col('cnt_deals_deny').desc())

In [271]:
deals_deny.cache()

DataFrame[product_cd_mmb: string, cnt_deals_deny: bigint]

In [272]:
deals_deny.show()

,product_cd_mmb,cnt_deals_deny
0,CREDCORPCARD,20285
1,SMARTCRED,4917
2,SMARTCARD,2055
3,EINVOICING,957
4,CORPCARD,912
5,SALARYPROJ,840
6,2216GARANTY,697
7,RKO,547
8,MOMENTBIZCARD,428
9,TELEMED,341


## Succeeded Deals

In [273]:
deals_s = \
deals.filter(f.col('appl_stage_name').like("%Закрыта/Заключена%")) \
     .filter(f.isnull(f.col('close_reason_type_name'))&(~f.isnull('product_cd_mmb'))
            ) \
     .select('inn', 
             'product_cd_mmb', 
             f.col('SUM_RUB').cast(IntegerType())
            ).orderBy(f.col('SUM_RUB').desc())

deals_succ = deals_s.groupBy('product_cd_mmb').agg(f.count('SUM_RUB').alias('cnt_deals_succ'),
                                                   f.sum('SUM_RUB').alias('total_sum_rub')
                                                  ).orderBy(f.col('cnt_deals_succ').desc())

In [274]:
deals_succ.cache()

DataFrame[product_cd_mmb: string, cnt_deals_succ: bigint, total_sum_rub: bigint]

In [275]:
deals_succ.show()

,product_cd_mmb,cnt_deals_succ,total_sum_rub
0,RKO,42582,288452
1,MINBALANCE,42506,6565099043
2,ENCASHSELF,36517,1946
3,DEPOSIT,30560,2678459350
4,SMSINFORM,15339,73789
5,MOMENTBIZCARD,14019,44
6,ACQUIRING,10571,87573912
7,CREDCORPCARD,9764,8427297
8,CORPCARD,7576,10184
9,SPPU,6167,35991


## MA_PRODUCT_DICT

In [329]:
prod_dct = hive.sql("select * from {}.{} where CRM_PRODUCT_ID is not Null".format(conn_schema, 'ma_dict_v_product_dict'))
prod_dct = prod_dct.filter("(PRODUCT_CD_MMB is not Null) and (PRODUCT_SHORT_NM is not Null)")

prod_dct.columns

['ID',
 'CRM_PRODUCT_ID',
 'PRODUCT_CRM_NM',
 'CRM_ACTIVE_FLG',
 'PRODUCT_CD_KSB',
 'PRODUCT_CD_RGS',
 'PRODUCT_CD_MMB',
 'PRODUCT_CD_ASUP',
 'PRODUCT_SHORT_NM',
 'PRODUCT_FULL_NM',
 'PARTNER_NM',
 'CHANNEL_SALE',
 'CHANNEL_KKSB_ISKRA',
 'CHANNEL_KKSB_EFS',
 'CHANNEL_RGS_ISKRA',
 'CHANNEL_ECO_EXPERT',
 'CHANNEL_MMB_MS',
 'CHANNEL_MMB_CKR',
 'CHANNEL_MMB_MKK',
 'CHANNEL_DZO',
 'AGG_FORMULA_KSB',
 'AGG_FORMULA_RGS',
 'CNT_DAY_DEAL',
 'PRODUCT_SUBGROUP',
 'PRODUCT_GROUP',
 'PRODUCT_TYPE',
 'PRODUCT_PRIORITY']

## MA_MMB_OFFER_NONTOP

In [336]:
nontop = hive.sql("select * from {}.{} where (INN is not Null) and (CREATE_DTTM >= timestamp('2021-04-01'))".format(conn_schema, 'MA_MMB_OFFER_NONTOP'))

In [337]:
def getProdCDFromId(product_dict_nm):
    def get_product(product_id, product_dict_nm):
        if product_id is None:
            return None

        for _id in product_dict_nm:
            if product_id == _id:
                return product_dict_nm[_id]

        return list([])

    return f.udf(lambda x: get_product(x,product_dict_nm), ArrayType(StringType()))

w2=Window.partitionBy('HASHINN', 'CRM_PRODUCT_ID').orderBy("COEF_PRODUCT_CD")

product_dict_sp = prod_dct.select('ID', 
                                  'CRM_PRODUCT_ID', 
                                  'PRODUCT_CD_MMB',
                                  'PRODUCT_SHORT_NM',
                                  'PRODUCT_GROUP', 
                                  'PRODUCT_SUBGROUP')

product_dict_pd = product_dict_sp.collect()
product_dict_nm = {row.ID: (row.CRM_PRODUCT_ID, 
                            row.PRODUCT_CD_MMB, 
                            row.PRODUCT_SHORT_NM,
                            row.PRODUCT_GROUP,
                            row.PRODUCT_SUBGROUP
                           ) for row in product_dict_pd}

nontop = nontop.withColumn('PROD_LST', getProdCDFromId(product_dict_nm)('PRODUCT_ID'))

nontop = nontop.select(['INN', f.col('COEF_PRODUCT_CD').alias("POTENTIAL")] +
                       [f.col('PROD_LST')[0].alias('CRM_PRODUCT_ID'), 
                        f.col('PROD_LST')[1].alias('PRODUCT_CD_MMB'),
                        f.col('PROD_LST')[2].alias('PRODUCT_SHORT_NM'),
                        f.col('PROD_LST')[3].alias('PRODUCT_GROUP'),
                        f.col('PROD_LST')[4].alias('PRODUCT_SUBGROUP')
                       ]).filter("PRODUCT_CD_MMB is not Null")

In [338]:
nontop.show()

,INN,POTENTIAL,CRM_PRODUCT_ID,PRODUCT_CD_MMB,PRODUCT_SHORT_NM,PRODUCT_GROUP,PRODUCT_SUBGROUP
0,5079005278,766.200012,1-2AZ4PW6S,2GIS_VIVESKA,2GIS-Вывеска,Экосистема,2GIS
1,5079005285,3832.199951,1-XNQFAFZ,DELOVAYA SREDA,Деловая среда-Подключение к Платформе Деловая ...,Экосистема,Деловая среда
2,5079006987,3832.199951,1-XNQFAFZ,DELOVAYA SREDA,Деловая среда-Подключение к Платформе Деловая ...,Экосистема,Деловая среда
3,5079006987,411.839996,1-MP71XI8,LYURIST,Юридические решения-Юрист для бизнеса,Экосистема,Юридические решения
4,507900751574,3929.560791,1-1Z85A72,OVERDRAFT,Овердрафт,Кредитование,Кредитование
5,507900751574,3928.852783,1-1GUC,ACQUIRING,Торговый эквайринг,Эквайринг,Эквайринг
6,507900751574,3832.199951,1-XNQFAFZ,DELOVAYA SREDA,Деловая среда-Подключение к Платформе Деловая ...,Экосистема,Деловая среда
7,507900751574,3714.423340,1-1GVC,CREDIT,Оборот. кредит-е,Кредитование,Кредитование
8,507900751574,1911.679565,1-HLPSULB,CREDCORPCARD,Кредитная бизнес-карта,Кредитование,Кредитная бизнес-карта
9,507900751574,848.261292,1-ASDFAMH,EVOTOR,Эвотор-Покупка смарт-терминала,Смарт-терминал,Эквайринг


In [340]:
mean_nontop = nontop.groupBy('CRM_PRODUCT_ID').agg(f.mean('POTENTIAL').cast(IntegerType()).alias('MEAN_POTENTIAL'),
                                                   f.first('PRODUCT_CD_MMB').alias('PRODUCT_CD_MMB'),
                                                   f.first('PRODUCT_SHORT_NM').alias('PRODUCT_SHORT_NM'),
                                                   f.first('PRODUCT_GROUP').alias('PRODUCT_GROUP'),
                                                   f.first('PRODUCT_SUBGROUP').alias('PRODUCT_SUBGROUP')
                                                  ).orderBy(
                                                            f.col("MEAN_POTENTIAL").desc()
                                                           )

mean_nontop.cache()

DataFrame[CRM_PRODUCT_ID: string, MEAN_POTENTIAL: int, PRODUCT_CD_MMB: string, PRODUCT_SHORT_NM: string, PRODUCT_GROUP: string, PRODUCT_SUBGROUP: string]

In [341]:
mean_nontop.count()

61

In [342]:
mean_nontop.show()

,CRM_PRODUCT_ID,MEAN_POTENTIAL,PRODUCT_CD_MMB,PRODUCT_SHORT_NM,PRODUCT_GROUP,PRODUCT_SUBGROUP
0,1-K3MF6Z8,2147483647,SMARTCARD,Смарт кредитная бизнес-карта,Кредитование,Кредитная бизнес-карта
1,1-79LMUVD,2147483647,FACTORING,Факторинг,Факторинг,Факторинг
2,1-EO0QXLI,2147483647,SMARTCRED,Смарт-кредит,Кредитование,Кредитование
3,1-1GVC,610759488,CREDIT,Оборот. кредит-е,Кредитование,Кредитование
4,1-1GW0,552343325,SALARYPROJ,ЗП (ФОТ),Зарплатные проекты,Зарплатные проекты
5,1-115HHZO,454389826,LEASING,Предзаявка в Сбербанк Лизинг,Кредитование,Лизинг
6,1-115HHYO,421709676,CORPCARD,Бизнес карта,Бизнес-карта,Бизнес-карта
7,1-1GUC,216869799,ACQUIRING,Торговый эквайринг,Эквайринг,Эквайринг
8,1-115HHZC,50552855,RKO,Открытие расчетного счета,РКО,РКО
9,1-CZ3R1Y,44477681,INSURANCE,Страхование,Страхование,Страхование


## OFFER_PRIORITY 

In [343]:
priority = hive.sql("select * from {}.{} where inn is not Null and load_dttm >= timestamp('2021-04-01')".format(conn_schema, 'OFFER_PRIORITY'))

In [344]:
priority_part = priority.select('INN', 
                                'CRM_PRODUCT_ID', 
                                f.col('offer_priority').alias("POTENTIAL") 
                                )

In [345]:
priority_part = priority_part.join(product_dict_sp, on='CRM_PRODUCT_ID', how='left_outer').select('INN', 
                                                                                                  'CRM_PRODUCT_ID', 
                                                                                                  'PRODUCT_CD_MMB',
                                                                                                  'PRODUCT_SHORT_NM',
                                                                                                  'PRODUCT_GROUP',
                                                                                                  'PRODUCT_SUBGROUP',
                                                                                                  'POTENTIAL').filter("PRODUCT_CD_MMB is not Null")

In [346]:
mean_priority = priority_part.groupBy('CRM_PRODUCT_ID').agg(f.mean('POTENTIAL').cast(IntegerType()).alias('MEAN_POTENTIAL'),
                                                            f.first('PRODUCT_CD_MMB').alias('PRODUCT_CD_MMB'),
                                                            f.first('PRODUCT_SHORT_NM').alias('PRODUCT_SHORT_NM'),
                                                            f.first('PRODUCT_GROUP').alias('PRODUCT_GROUP'),
                                                            f.first('PRODUCT_SUBGROUP').alias('PRODUCT_SUBGROUP')
                                                           ).orderBy(f.col("MEAN_POTENTIAL").desc())

mean_priority.cache()

DataFrame[CRM_PRODUCT_ID: string, MEAN_POTENTIAL: int, PRODUCT_CD_MMB: string, PRODUCT_SHORT_NM: string, PRODUCT_GROUP: string, PRODUCT_SUBGROUP: string]

In [347]:
mean_priority.cache()

DataFrame[CRM_PRODUCT_ID: string, MEAN_POTENTIAL: int, PRODUCT_CD_MMB: string, PRODUCT_SHORT_NM: string, PRODUCT_GROUP: string, PRODUCT_SUBGROUP: string]

In [348]:
mean_priority.count()

49

## Union Offers

In [349]:
union_offers = mean_priority.unionAll(mean_nontop).select('CRM_PRODUCT_ID', 
                                                          'PRODUCT_CD_MMB', 
                                                          'PRODUCT_SHORT_NM',
                                                          'PRODUCT_GROUP',
                                                          'PRODUCT_SUBGROUP',
                                                          'MEAN_POTENTIAL')
union_offers = union_offers.drop_duplicates()

In [350]:
union_offers = union_offers.join(deals_succ, on=['PRODUCT_CD_MMB'], how='left_outer').select(
                                                                                             'CRM_PRODUCT_ID',
                                                                                             'PRODUCT_CD_MMB',
                                                                                             'PRODUCT_SHORT_NM',
                                                                                             'PRODUCT_GROUP',
                                                                                             'PRODUCT_SUBGROUP',    
                                                                                             'MEAN_POTENTIAL',
                                                                                             'cnt_deals_succ',
                                                                                             'total_sum_rub'          
                                                                                             )


union_offers = union_offers.join(deals_deny, on=['PRODUCT_CD_MMB'], how='left_outer').select( 
                                                                                             'CRM_PRODUCT_ID',
                                                                                             'PRODUCT_CD_MMB',
                                                                                             'PRODUCT_SHORT_NM',
                                                                                             'PRODUCT_GROUP',
                                                                                             'PRODUCT_SUBGROUP',    
                                                                                             'MEAN_POTENTIAL',
                                                                                             'cnt_deals_deny',
                                                                                             'cnt_deals_succ',
                                                                                             'total_sum_rub'
                                                                                             )

union_offers = union_offers.drop_duplicates()

In [351]:
fin = union_offers.groupBy('CRM_PRODUCT_ID').agg( f.first('PRODUCT_CD_MMB').alias('PRODUCT_CD_MMB'),
                                                  f.first('PRODUCT_SHORT_NM').alias('PRODUCT_SHORT_NM'),
                                                  f.first('PRODUCT_GROUP').alias('PRODUCT_GROUP'),
                                                  f.first('PRODUCT_SUBGROUP').alias('PRODUCT_SUBGROUP'),
                                                  f.first('MEAN_POTENTIAL').alias('MEAN_POTENTIAL'),
                                                  f.sum('cnt_deals_succ').alias('cnt_deals_succ'.upper()),
                                                  f.sum('cnt_deals_deny').alias('cnt_deals_deny'.upper()),
                                                  f.sum('total_sum_rub').alias('total_sum_rub'.upper()),
                                                  ).orderBy(f.col("MEAN_POTENTIAL").desc(),
                                                            f.col("cnt_deals_succ").desc()
                                                           )

In [352]:
df = fin.filter(~f.isnull('cnt_deals_succ')).toPandas()

In [353]:
df.head()

,CRM_PRODUCT_ID,PRODUCT_CD_MMB,PRODUCT_SHORT_NM,PRODUCT_GROUP,PRODUCT_SUBGROUP,MEAN_POTENTIAL,CNT_DEALS_SUCC,CNT_DEALS_DENY,TOTAL_SUM_RUB
0,1-2X6RWK2B,SPASIBO_BUSINESS,"Программа лояльности ""Спасибо Бизнес""",Экосистема,СберСпасибо,2147483647,2284,18.0,0
1,1-EO0QXLI,SMARTCRED,Смарт-кредит,Кредитование,Кредитование,2147483647,1198,4917.0,1351566
2,1-1GVC,CREDIT,Оборот. кредит-е,Кредитование,Кредитование,2147483647,694,300.0,164146022
3,1-K3MF6Z8,SMARTCARD,Смарт кредитная бизнес-карта,Кредитование,Кредитная бизнес-карта,2147483647,691,2055.0,343246
4,1-2UI5D8E1,DELOVAYA_SREDA-FORUM_SBER,Деловая среда-Форум Сбер Бизнес Live,Экосистема,Деловая среда,2147483647,74,42.0,1002


In [354]:
df.to_excel("/opt/workspace/ektov1-av_ca-sbrf-ru/notebooks/site_idprod_deals_rank.xlsx", index = False, encoding='utf8')